In [1]:
from tsrrt.collision import PandaCollision
import pybullet as p
import pybullet_data
import numpy as np
from spatial_math_mini import *
from collections.abc import Sequence
from abc import ABC, abstractmethod
import datetime

#local
from tsrrt.data_structure import Node, StateSpace, Tree
from tsrrt.util import *
from tsrrt.RRT import RRT

pybullet build time: Oct 11 2021 20:59:39


In [2]:
uid = p.connect(p.GUI)
checker = PandaCollision(p, uid)

In [3]:
pos_ll = [-1, -1, 0]
pos_ul = [1, 1, 1]
goal_pos = [0.5, 0.5, 0.3]
pos = np.empty(3)
for i in range(3):
    pos[i] = np.random.normal(loc=goal_pos[i], scale=(pos_ul[i]-pos_ll[i])/4)

In [4]:
ss = StateSpace(Node)

#init
checker.set_home_positions()
(pos, ori), q = checker.get_ee_pose(), checker.get_arm_positions()
start = Node(pos, ori, q)
tree = Tree(start, state_space=ss)

### test control

In [5]:
goal = ss.random()

In [6]:
rrt = RRT(start, goal, checker, bias="normal", debug=True)

In [7]:
goal = rrt.sample_free_space()
rrt._goal = goal
view_node(goal, color="r")

In [8]:
success, plan = rrt.plan()

goal!


In [9]:
rrt._tree._num

151

In [151]:
i=0

In [186]:
i += 1
checker.set_joint_positions(plan[i].q)

IndexError: list index out of range

In [111]:
pos = [p.pos for p in plan]
ori = [p.ori for p in plan]

In [113]:
pos

[array([ 5.54500000e-01, -3.21955792e-12,  5.19500000e-01]),
 array([ 0.52305618, -0.02851684,  0.39818893]),
 array([0.53623008, 0.07809318, 0.51531339]),
 array([0.5043065 , 0.04709864, 0.39777575]),
 array([0.54313713, 0.16395044, 0.4493312 ]),
 array([0.45947188, 0.01171518, 0.29299795]),
 array([ 0.39111437, -0.05987364,  0.24862254]),
 array([0.36586533, 0.0500737 , 0.30076289]),
 array([0.32432457, 0.03998683, 0.24277977]),
 array([0.33599836, 0.10209956, 0.27231453]),
 array([0.27242628, 0.1753539 , 0.22107631]),
 array([0.20011648, 0.15230781, 0.18453188]),
 array([0.17817987, 0.24925906, 0.25295769]),
 array([0.16628472, 0.33653309, 0.33826698]),
 array([0.11986903, 0.30741043, 0.25107547]),
 array([0.0558843 , 0.48251962, 0.58003019]),
 array([0.02270765, 0.48519216, 0.55809947]),
 array([0.16044241, 0.49878542, 0.61975167]),
 array([0.16160608, 0.50668433, 0.54768082]),
 array([0.08897719, 0.48811269, 0.53106922]),
 array([0.15281549, 0.53195701, 0.4546764 ]),
 array([0.102

In [17]:
node = rrt.control(nn, goal)
if node is not None:
    checker.set_joint_positions(node.q)

dist:0.023245002251242666, curr:0.019042754335967832
dist:0.023245002251242666, curr:0.01561377009946651
dist:0.023245002251242666, curr:0.01281404214903025
dist:0.023245002251242666, curr:0.010526422050576365
dist:0.023245002251242666, curr:0.008655720596282923
dist:0.023245002251242666, curr:0.007124622567633395
dist:0.023245002251242666, curr:0.005870330045603882
dist:0.023245002251242666, curr:0.004841823397040523
dist:0.023245002251242666, curr:0.003997634421956747
dist:0.023245002251242666, curr:0.003304039621493834
dist:0.023245002251242666, curr:0.0027335960553265667
dist:0.023245002251242666, curr:0.002263955579507995
dist:0.023245002251242666, curr:0.0018769047920908588
dist:0.023245002251242666, curr:0.001557587728438294
dist:0.023245002251242666, curr:0.0012938764027691937
dist:0.023245002251242666, curr:0.00107586090631307
dist:0.023245002251242666, curr:0.0008954361707611535
dist:0.023245002251242666, curr:0.0007459668926179393
dist:0.023245002251242666, curr:0.0006220156

KeyboardInterrupt: 

In [16]:
if node is not None:
    node = rrt.control(node, goal)
    checker.set_joint_positions(node.q)
    print(ss.distance(node, goal, is_weighted=True))
else:
    print("control failed")

dist:0.0012727346752713835, curr:0.0010242734124335964
dist:0.0012727346752713835, curr:0.0008243207813463898
dist:0.0012727346752713835, curr:0.0006634063027955989
dist:0.0012727346752713835, curr:0.0005339080359256045
dist:0.0012727346752713835, curr:0.00042969198518822126
dist:0.0012727346752713835, curr:0.00034582175021569307
dist:0.0012727346752713835, curr:0.00027832477567523437
dist:0.0012727346752713835, curr:0.00022400419189312623
dist:0.0012727346752713835, curr:0.00018028736790526165
dist:0.0012727346752713835, curr:0.00014510402274753084
0.00014510402274753084


In [131]:
node_next = rrt.control(node, goal)
checker.set_joint_positions(node.q)
print(ss.distance(node, node_next, is_weighted=True))
node = node_next

0.04079811994040256


### weighting effect in Nearest

In [7]:
for i in range(15):
    node = ss.random()
    tree.add(node, start)
    view_node(node, color="k")

In [9]:
node = ss.random()
view_node(node)
nn_w = tree.nearest(node)
nn = tree.nearest(node, is_weighted=False)
view_node(nn_w, "r")
view_node(nn, "g")